In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/NLP/revpre.csv')


In [4]:
df['label'].value_counts()

1    19314
0     4172
Name: label, dtype: int64

In [ ]:
df.shape

(23486, 6)

In [6]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Review,Rating,label,Positive Feedback Count
0,0,0,NaN,4,1,0
1,1,1,NaN,5,1,4
2,2,2,NaN,3,0,0
3,3,3,NaN,5,1,0
4,4,4,NaN,5,1,6


In [ ]:
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load("en_core_web_lg")

2023-04-12 09:12:07.584362: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-12 09:12:10.037120: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
nlp("feel today legday jelly ache gym").vector

array([ 1.68045175e+00,  8.15226734e-01, -1.20932662e+00, -5.10283351e-01,
       -8.26773345e-01, -1.25090659e+00, -3.69484335e-01,  1.86937511e+00,
       -9.36916649e-01,  1.23137343e+00,  2.54665017e+00,  7.98018277e-01,
       -1.26332445e-02,  2.32047486e+00,  2.38205504e+00, -2.12688828e+00,
        6.24416649e-01, -8.75853360e-01, -1.60570002e+00, -1.84330022e+00,
        9.68449891e-01,  1.58537996e+00,  2.72878408e-01, -2.08236146e+00,
       -2.49944970e-01, -4.17616695e-01, -4.86057848e-01, -1.17727339e+00,
        1.03483546e+00,  1.25202835e+00,  1.51832831e+00, -6.13253295e-01,
        6.02168823e-03, -2.07076693e+00,  1.55910015e+00,  1.34525168e+00,
        1.07888329e+00,  2.28481007e+00,  4.24191743e-01,  1.02048302e+00,
       -9.88665044e-01,  1.40706670e+00,  7.63133347e-01, -3.20828319e-01,
        2.57764280e-01,  1.49299657e+00, -4.64185067e-02, -2.08144522e+00,
       -6.32928252e-01, -5.26746690e-01,  2.25669503e+00,  3.14990687e-03,
        1.09429669e+00, -

In [ ]:
df['Review']=str(df['Review'])
df['vector']=df['Review'].apply(lambda text:nlp(text).vector)

In [ ]:
df.head()

,Unnamed: 0.1,Unnamed: 0,Review,Rating,label,Positive Feedback Count,vector
0,0,0,0 NaN\n1 NaN\n2 NaN\n3 ...,4,1,0,"[-0.07847039, -1.5175925, 1.4045286, 1.5083058..."
1,1,1,0 NaN\n1 NaN\n2 NaN\n3 ...,5,1,4,"[-0.07847039, -1.5175925, 1.4045286, 1.5083058..."
2,2,2,0 NaN\n1 NaN\n2 NaN\n3 ...,3,0,0,"[-0.07847039, -1.5175925, 1.4045286, 1.5083058..."
3,3,3,0 NaN\n1 NaN\n2 NaN\n3 ...,5,1,0,"[-0.07847039, -1.5175925, 1.4045286, 1.5083058..."
4,4,4,0 NaN\n1 NaN\n2 NaN\n3 ...,5,1,6,"[-0.07847039, -1.5175925, 1.4045286, 1.5083058..."


In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.vector.values, df.label, test_size=0.2)
import numpy as np
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


clf = GaussianNB()
clf.fit(scaled_train_embed, y_train)
from sklearn.metrics import classification_report

y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.17      1.00      0.29       803
           1       0.00      0.00      0.00      3895

    accuracy                           0.17      4698
   macro avg       0.09      0.50      0.15      4698
weighted avg       0.03      0.17      0.05      4698



/usr/local/lib/python3.9/dist-packages/sklearn/naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.9/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in true_divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.vector, df.label, test_size=0.2)
import numpy as np
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


clf = RandomForestClassifier()
clf.fit(scaled_train_embed, y_train)
from sklearn.metrics import classification_report

y_pred = clf.predict(scaled_test_embed)

print(classification_report(y_test, y_pred))




              precision    recall  f1-score   support

           0       0.00      0.00      0.00       852
           1       0.82      1.00      0.90      3846

    accuracy                           0.82      4698
   macro avg       0.41      0.50      0.45      4698
weighted avg       0.67      0.82      0.74      4698



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
